# Screen Time Mental Health EDA

In [14]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

Get the dataframe

In [15]:
df = pd.read_csv("data/maps-synthetic-data-v1.1.csv", index_col="X")
df = df.drop(df.columns[0], axis=1)
df.head()

,flag,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,...,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
X,,,,,,,,,,,,,,,,,,,,,
1,synthetic,NaN,3.0,30,59.294132,181.602831,107.0,NaN,NaN,Any at all,...,0.0,Yes,Yes,often,No,"Yes, Some Days",Other,"Yes, Every Day",Female,A
2,synthetic,NaN,9.0,26,NaN,NaN,85.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,B
3,synthetic,NaN,3.0,24,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Yes, Some Days","Yes, Every Day",Other,Male,A
4,synthetic,No,NaN,22,49.812426,160.224186,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,Male,A
5,synthetic,No,8.0,31,62.270030,191.703227,132.0,Yes,NaN,Not at all,...,NaN,NaN,NaN,often,No,"Yes, Some Days","Yes, Every Day","Yes, Every Day",Female,A


In [16]:
df.columns

Index(['flag', 'comp_bed_9', 'mat_dep', 'mat_age', 'weight_16', 'height_16',
       'iq', 'comp_noint_bed_16', 'comp_int_bed_16', 'talk_phon_wend',
       'text_wend', 'talk_mob_wend', 'comp_wend', 'musi_wend', 'read_wend',
       'work_wend', 'alon_wend', 'draw_wend', 'play_wend', 'tv_wend',
       'out_win_wend', 'out_sum_wend', 'tran_wend', 'talk_phon_week',
       'text_week', 'talk_mob_week', 'comp_week', 'musi_week', 'read_week',
       'work_week', 'alon_week', 'draw_week', 'play_week', 'tv_week',
       'out_win_week', 'out_sum_week', 'tran_week', 'pat_pres_10',
       'pat_pres_8', 'pat_pres', 'num_home', 'mat_anx_1', 'mat_anx_18m',
       'mat_anx_8m', 'agg_score', 'emot_cruel', 'phys_cruel', 'mat_anx_0m',
       'pat_ses', 'mat_ses', 'pat_edu', 'mat_edu', 'parity', 'dep_band_15',
       'dep_band_13', 'dep_band_10', 'dep_band_07', 'anx_band_15',
       'anx_band_13', 'anx_band_10', 'anx_band_07', 'exercise', 'child_bull',
       'phone_14_wend', 'phone_14_week', 'musi_13', '

### Correlation

Look at the correlation between each feature.

In [17]:
df.corr()

,mat_dep,weight_16,height_16,iq,agg_score,parity,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score
mat_dep,1.000000,0.022857,-0.037471,-0.076213,-0.399370,0.072405,0.042720,0.049515,0.044806,0.036001,0.042168
weight_16,0.022857,1.000000,0.506162,-0.010114,-0.023589,0.023191,-0.045174,-0.032918,-0.014801,-0.044244,-0.014034
height_16,-0.037471,0.506162,1.000000,0.108182,-0.006108,-0.000875,-0.058494,-0.042952,-0.016259,-0.049831,-0.034491
iq,-0.076213,-0.010114,0.108182,1.000000,0.064654,-0.075945,-0.002553,-0.011164,-0.003665,-0.024857,0.007587
agg_score,-0.399370,-0.023589,-0.006108,0.064654,1.000000,-0.040754,-0.015039,-0.026265,-0.062852,-0.036543,-0.040472
parity,0.072405,0.023191,-0.000875,-0.075945,-0.040754,1.000000,0.045679,0.036739,0.022520,0.039167,0.036022
secd_diag,0.042720,-0.045174,-0.058494,-0.002553,-0.015039,0.045679,1.000000,0.761506,0.517815,0.544721,0.617543
prim_diag,0.049515,-0.032918,-0.042952,-0.011164,-0.026265,0.036739,0.761506,1.000000,0.324851,0.632179,0.725219
panic_score,0.044806,-0.014801,-0.016259,-0.003665,-0.062852,0.022520,0.517815,0.324851,1.000000,0.200793,0.238634
dep_thoughts,0.036001,-0.044244,-0.049831,-0.024857,-0.036543,0.039167,0.544721,0.632179,0.200793,1.000000,0.610364


Note: 
- not all feature included as many features are categorical (string) type
- NaN types ignored

## Categorical feature handling

Look at the feature data types

In [18]:
for dtype in df.dtypes:
    print(dtype)

object
object
float64
object
float64
float64
float64
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
float64
object
object
object
object
object
object
object
float64
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
float64
float64
float64
float64
float64
object
object
object
object
object
object
object
object
object


Check the cataegorical feature categories.

In [19]:
for col in df:
    if df[col].dtype == "object":
        print(f"{col}: {df[col].unique()}\n")

flag: ['synthetic']

comp_bed_9: [nan 'No' 'Yes']

mat_age: ['30' '26' '24' '22' '31' '36' '27' '21' '32' '< 16' '33' '25' '16' '37'
 '34' '35' '18' '29' '23' '20' '17' '28' '38' nan '40' '19' '39' '41'
 '>43' '42' '43']

comp_noint_bed_16: [nan 'Yes']

comp_int_bed_16: [nan 'Yes']

talk_phon_wend: ['Any at all' nan 'Not at all']

text_wend: ['Less than 1 hour' nan 'Not at all' '3 or more hours' '1-2 hours']

talk_mob_wend: ['Less than 1 hour' nan 'Not at all' '1 or more hours']

comp_wend: ['1-2 hours' nan 'Less than 1 hour' '3 or more hours' 'Not at all']

musi_wend: ['Not at all' nan 'Any at all']

read_wend: ['Less than 1 hour' nan '1 or more hours' 'Not at all']

work_wend: ['3 or more hours' nan '1-2 hours' 'Less than 1 hour']

alon_wend: ['1-2 hours' nan '3 or more hours' 'Less than 1 hour']

draw_wend: ['Not at all' nan 'Less than 1 hour' '1 or more hours']

play_wend: ['3 or more hours' nan 'Less than 3 hours']

tv_wend: ['1-2 hours' nan '3 or more hours' 'Less than 1 hour' 'N

Create a dataframe for the categorical types.

In [20]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df

,flag,comp_bed_9,mat_age,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,...,has_dep_diag,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
X,,,,,,,,,,,,,,,,,,,,,
1,synthetic,NaN,30,NaN,NaN,Any at all,Less than 1 hour,Less than 1 hour,1-2 hours,Not at all,...,No ICD-10 diagnosis of depression,Yes,Yes,often,No,"Yes, Some Days",Other,"Yes, Every Day",Female,A
2,synthetic,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,B
3,synthetic,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Yes, Some Days","Yes, Every Day",Other,Male,A
4,synthetic,No,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,Male,A
5,synthetic,No,31,Yes,NaN,Not at all,Not at all,Not at all,Less than 1 hour,Not at all,...,NaN,NaN,NaN,often,No,"Yes, Some Days","Yes, Every Day","Yes, Every Day",Female,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13730,synthetic,NaN,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Yes, Every Day","Yes, Every Day","Yes, Every Day",Female,A
13731,synthetic,No,29,NaN,NaN,Not at all,Less than 1 hour,Less than 1 hour,1-2 hours,Any at all,...,NaN,NaN,NaN,NaN,No,"Yes, Every Day",Other,Other,Male,A
13732,synthetic,NaN,37,NaN,NaN,Any at all,1-2 hours,1 or more hours,1-2 hours,Not at all,...,NaN,NaN,NaN,often,No,Other,Other,Other,Male,A


We can create a feature map using a dictionary to apply feature conversion to feature columns.

In [21]:
feature_map = {    
    "comp_bed_9": {"No": 0, "Yes": 1},
    "mat_age": {"30": 30, "26": 26, "24": 24, "22": 22, "31": 31, "36": 36, "27": 27, "21": 21, "32": 32, "< 16": 15, "33": 33, "25": 25, "16": 16, "37": 37, "34": 34, "35": 35, "18": 18, "29": 29, "23": 23, "20": 20, "17": 17, "28": 28, "38": 38, "40": 40, "19": 19, "39": 39, "41": 31, ">43": 44, "42": 42, "43": 43},
    "comp_noint_bed_16": {"No": 0, "Yes": 1},
    "comp_int_bed_16": {"No": 0, "Yes": 1},
    "talk_phon_wend": {"Not at all": 0, "Any at all": 1},
    "text_wend": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "talk_mob_wend": {"Not at all": 0, "Less than 1 hour": 1, "1 or more hours": 2},
    "comp_wend": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "musi_wend": {"Not at all": 0, "Any at all": 1},
    "read_wend": {"Not at all": 0, "Less than 1 hour": 1, "1 or more hours": 2},
    "work_wend": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "alon_wend": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "draw_wend": {"Not at all": 0, "Less than 1 hour": 1, "1 or more hours": 2},
    "play_wend": {"Less than 3 hours": 0, "3 or more hours": 1},
    "tv_wend": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "out_win_wend": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "out_sum_wend": {"Less than 3 hours": 0, "3 or more hours": 1},
    "tran_wend": {"Not at all": 0, "Less than 1 hour": 1, "1 or more hours": 2},
    "talk_phon_week": {"Not at all": 0, "Any at all": 1},
    "text_week": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "talk_mob_week": {"Not at all": 0, "Less than 1 hour": 1, "1 or more hours": 2},
    "comp_week": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "musi_week": {"Not at all": 0, "Any at all": 1},
    "read_week": {"Not at all": 0, "Less than 1 hour": 1, "1 or more hours": 2},
    "work_week": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "alon_week": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "draw_week": {"Not at all": 0, "Less than 1 hour": 1, "1 or more hours": 2},
    "play_week": {"Less than 3 hours": 0, "3 or more hours": 1},
    "tv_week": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "out_win_week": {"Not at all": 0, "Less than 1 hour": 1, "1-2 hours": 2, "3 or more hours": 3},
    "out_sum_week": {"Less than 3 hours": 0, "3 or more hours": 1},
    "tran_week": {"Not at all": 0, "Less than 1 hour": 1, "1 or more hours": 2},
    "pat_pres_10": {"No": 0, "Yes": 1},
    "pat_pres_8": {"No": 0, "Yes": 1},
    "pat_pres": {"No": 0, "Yes": 1},
    "num_home": {"1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9 or more": 9},
    "mat_anx_1": {"No": 0, "Yes": 1},
    "mat_anx_18m": {"No": 0, "Yes": 1},
    "mat_anx_8m": {"No": 0, "Yes": 1},
    "emot_cruel": {"No": 0, "Yes": 1},
    "phys_cruel": {"No": 0, "Yes": 1},
    "mat_anx_0m": {"No": 0, "Yes": 1},
    "pat_ses": {"i": 1, "ii": 2, "III (manual)": 3, "III (non-manual)": 4, "iv": 5, "v": 6, "Armed forces": 7},
    "mat_ses": {"i": 1, "ii": 2, "III (manual)": 3, "III (non-manual)": 4, "iv": 5, "v": 6, "Armed forces": 7},
    "pat_edu": {"cse": 0, "O level": 1, "A level": 2, "Vocational": 3, "Degree": 4},
    "mat_edu": {"CSE/None": 0,  "O level": 1, "A level": 2, "Vocational": 3, "Degree": 4},
    "dep_band_15": {"<0.1%": 0, "~0.5%": 1, "~3%": 2, "~15%": 3, "~50%": 4, ">70%": 5},
    "dep_band_13": {"<0.1%": 0, "~0.5%": 1, "~3%": 2, "~15%": 3, "~50%": 4, ">70%": 5},
    "dep_band_10": {"<0.1%": 0, "~0.5%": 1, "~3%": 2, "~15%": 3, "~50%": 4, ">70%": 5},
    "dep_band_07": {"<0.1%": 0, "~0.5%": 1, "~3%": 2, "~15%": 3, "~50%": 4, ">70%": 5},
    "anx_band_15": {"<0.1%": 0, "~0.5%": 1, "~3%": 2, "~15%": 3, "~50%": 4, ">70%": 5},
    "anx_band_13": {"<0.1%": 0, "~0.5%": 1, "~3%": 2, "~15%": 3, "~50%": 4, ">70%": 5},
    "anx_band_10": {"<0.1%": 0, "~0.5%": 1, "~3%": 2, "~15%": 3, "~50%": 4, ">70%": 5},
    "anx_band_07": {"<0.1%": 0, "~0.5%": 1, "~3%": 2, "~15%": 3, "~50%": 4, ">70%": 5},
    "exercise": {"Never": 0, "Less than once a month": 1, "1-3 times a month": 2, "1-4 times a week": 3, "5 or more times a week": 4},
    "child_bull": {"No": 0, "Yes": 1},
    "phone_14_wend": {"Not at all": 0, "< 1 hour": 1, "1 or more hours": 2},
    "phone_14_week": {"Not at all": 0, "< 1 hour": 1, "1 or more hours": 2},
    "musi_13": {"No": 0, "Yes": 1},
    "tv_bed_9": {"No": 0, "Yes": 1},
    "own_mob": {"No": 0, "Yes": 1},
    "has_dep_diag": {" No ICD-10 diagnosis of depression": 0, "Yes ICD-10 diagnosis of depression": 1},
    "comp_house": {"No": 0, "Yes": 1},
    "tv_bed_16": {"No": 0, "Yes": 1},
    "creat_14": {"sometimes": 0, "often": 1},
    "comp_games": {"No": 0, "Yes": 1},
    "fam_tv_eve": {"Other": 0, "Yes, Some Days": 1, "Yes, Every Day": 2},
    "fam_tv_aft": {"Other": 0, "Yes, Some Days": 1, "Yes, Every Day": 2},
    "fam_tv_mor": {"Other": 0, "Yes, Some Days": 1, "Yes, Every Day": 2},
    "sex": {"Female": 0, "Male": 1},
    "birth_order": {"A": 0, "B": 1},
}

Map feature dictionary to dataframe

In [22]:
obj_df = obj_df.replace(feature_map)
obj_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13734 entries, 1 to 13734
Data columns (total 72 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   flag               13734 non-null  object 
 1   comp_bed_9         8174 non-null   float64
 2   mat_age            13326 non-null  float64
 3   comp_noint_bed_16  471 non-null    float64
 4   comp_int_bed_16    2570 non-null   float64
 5   talk_phon_wend     4850 non-null   float64
 6   text_wend          4841 non-null   float64
 7   talk_mob_wend      4834 non-null   float64
 8   comp_wend          4839 non-null   float64
 9   musi_wend          4831 non-null   float64
 10  read_wend          4834 non-null   float64
 11  work_wend          4833 non-null   float64
 12  alon_wend          4850 non-null   float64
 13  draw_wend          4842 non-null   float64
 14  play_wend          4850 non-null   float64
 15  tv_wend            4853 non-null   float64
 16  out_win_wend       484

We do not need to flag feature as all data is synthetic so we shall drop it.

In [23]:
obj_df.drop("flag", axis=1, inplace=True)
obj_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13734 entries, 1 to 13734
Data columns (total 71 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   comp_bed_9         8174 non-null   float64
 1   mat_age            13326 non-null  float64
 2   comp_noint_bed_16  471 non-null    float64
 3   comp_int_bed_16    2570 non-null   float64
 4   talk_phon_wend     4850 non-null   float64
 5   text_wend          4841 non-null   float64
 6   talk_mob_wend      4834 non-null   float64
 7   comp_wend          4839 non-null   float64
 8   musi_wend          4831 non-null   float64
 9   read_wend          4834 non-null   float64
 10  work_wend          4833 non-null   float64
 11  alon_wend          4850 non-null   float64
 12  draw_wend          4842 non-null   float64
 13  play_wend          4850 non-null   float64
 14  tv_wend            4853 non-null   float64
 15  out_win_wend       4849 non-null   float64
 16  out_sum_wend       482

All types are now numerical, we shall ensure all types are the same (`float64`).

In [24]:
obj_df["sex"] = obj_df["sex"].astype("float64")
obj_df["birth_order"] = obj_df["birth_order"].astype("float64")
obj_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13734 entries, 1 to 13734
Data columns (total 71 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   comp_bed_9         8174 non-null   float64
 1   mat_age            13326 non-null  float64
 2   comp_noint_bed_16  471 non-null    float64
 3   comp_int_bed_16    2570 non-null   float64
 4   talk_phon_wend     4850 non-null   float64
 5   text_wend          4841 non-null   float64
 6   talk_mob_wend      4834 non-null   float64
 7   comp_wend          4839 non-null   float64
 8   musi_wend          4831 non-null   float64
 9   read_wend          4834 non-null   float64
 10  work_wend          4833 non-null   float64
 11  alon_wend          4850 non-null   float64
 12  draw_wend          4842 non-null   float64
 13  play_wend          4850 non-null   float64
 14  tv_wend            4853 non-null   float64
 15  out_win_wend       4849 non-null   float64
 16  out_sum_wend       482

Now we can combine the new mapped features with the numerical features.

In [25]:
data_df = df.copy()
data_df.drop("flag", axis=1, inplace=True)
data_df[obj_df.columns] = obj_df
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13734 entries, 1 to 13734
Data columns (total 82 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   comp_bed_9         8174 non-null   float64
 1   mat_dep            11145 non-null  float64
 2   mat_age            13326 non-null  float64
 3   weight_16          5351 non-null   float64
 4   height_16          5364 non-null   float64
 5   iq                 7288 non-null   float64
 6   comp_noint_bed_16  471 non-null    float64
 7   comp_int_bed_16    2570 non-null   float64
 8   talk_phon_wend     4850 non-null   float64
 9   text_wend          4841 non-null   float64
 10  talk_mob_wend      4834 non-null   float64
 11  comp_wend          4839 non-null   float64
 12  musi_wend          4831 non-null   float64
 13  read_wend          4834 non-null   float64
 14  work_wend          4833 non-null   float64
 15  alon_wend          4850 non-null   float64
 16  draw_wend          484

---

Feature map automation attempt

In [26]:
# nny = ["No", "Yes"]

# for col in obj_df:

#     unique_list = obj_df[col].unique().tolist()

# ##

# test0 = ["Yes", "No", "NaN"]
# test1 = ["NaN", "No", "Yes"]

# if Counter(nny) == Counter(test0):
#     print("yes")

# ##

# obj_df["comp_bed_9"] = obj_df["comp_bed_9"].astype("category")
# obj_df["comp_bed_9_comp"] = obj_df["comp_bed_9"].cat.codes

# obj_df[["comp_bed_9", "comp_bed_9_comp"]]

# ##

# obj_df["tv_bed_16"] = obj_df["tv_bed_16"].astype("category")
# obj_df["tv_bed_16_comp"] = obj_df["tv_bed_16"].cat.codes

# obj_df[["tv_bed_16", "tv_bed_16_comp"]]
    